In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.9 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score as calculate_f1_score


# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
edge_index_train = torch.load('/content/drive/MyDrive/PROJECT/edge_index.pt')
edge_index_test = torch.load('/content/drive/MyDrive/PROJECT/edge_index_test.pt')
edge_index_val = torch.load('/content/drive/MyDrive/PROJECT/edge_index_val.pt')

features_file_train = '/content/drive/MyDrive/PROJECT/feature_matrix_train.txt'
X_train = np.loadtxt(features_file_train)
features_file_test = '/content/drive/MyDrive/PROJECT/feature_matrix_test.txt'
X_test = np.loadtxt(features_file_test)
features_file_val = '/content/drive/MyDrive/PROJECT/feature_matrix_val.txt'
X_val = np.loadtxt(features_file_val)

labels_test = pd.read_csv("/content/drive/MyDrive/PROJECT/test_filtered.csv")
labels_train = pd.read_csv("/content/drive/MyDrive/PROJECT/train_filtered.csv")
labels_val = pd.read_csv("/content/drive/MyDrive/PROJECT/val_filtered.csv")

y_train = torch.tensor(labels_train['label'].values, dtype=torch.long).to(device)
y_val = torch.tensor(labels_val['label'].values, dtype=torch.long).to(device)
y_true = torch.tensor(labels_test['label'].values, dtype=torch.long).to(device)

# Preprocess features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)

X_test_scaled = scaler.transform(X_test)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

X_val_scaled = scaler.transform(X_val)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)

class GCNNet(nn.Module):
    def __init__(self, num_features, hidden_dim, output_dim, num_layers=1):
        super(GCNNet, self).__init__()


        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


# Define optimizer and loss function
model = GCNNet(num_features=X_train.shape[1], hidden_dim=8, output_dim=13).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()  # Define criterion here





# Training
def train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100):
    best_val_loss = float('inf')
    best_val_acc = 0.0
    current_patience = 0
    train_losses = []
    val_losses = []

    train_f1_scores = []  # Initialize list for training F1 scores
    epochss = []

    for epoch in range(epochs):
        epochss.append(epoch)
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train, edge_train)
        loss = criterion(outputs, y_train)
        train_losses.append(loss.cpu().item())
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted_train = torch.max(outputs, 1)
        train_acc = torch.sum(predicted_train == y_train).item() / len(y_train)
        # Calculate training F1 score
        train_f1 = calculate_f1_score(y_train.cpu().numpy(), predicted_train.cpu().numpy(), average='weighted')


        # Save training F1 score
        train_f1_scores.append(train_f1)



        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val, edge_val)
            val_loss = criterion(val_outputs, y_val)
            val_losses.append(val_loss)
            # Compute validation accuracy
            _, predicted_val = torch.max(val_outputs, 1)
            val_acc = torch.sum(predicted_val == y_val).item() / len(y_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pt')
            current_patience = 0
        else:
            current_patience += 1
            if current_patience >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}, Train Acc: {train_acc:.4f},train F1-score:{train_f1:.4f} Val Loss: {val_loss.item()}, Val Acc: {val_acc:.4f}')

    np.savetxt('/content/drive/MyDrive/PROJECT/train_f1_scores_GCN.txt',train_f1_scores)
    np.savetxt('/content/drive/MyDrive/PROJECT/train_loss_GCN.txt', train_losses)
    np.savetxt('/content/drive/MyDrive/PROJECT/epochs_GCN.txt', epochss)






# Convert data to appropriate format
edge_train = edge_index_train.to(device)
edge_val = edge_index_val.to(device)
edge_test = edge_index_test.to(device)

# Train the model
train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100)

# Load the best model
model.load_state_dict(torch.load('best_model.pt'))

# Testing
model.eval()
with torch.no_grad():
    test_outputs = model(X_test, edge_test)
    test_loss = criterion(test_outputs, y_true)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = torch.sum(predicted == y_true).item() / len(y_true)

print(f'Test Loss: {test_loss.item()}, Test Accuracy: {accuracy}')

from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), predicted.cpu().numpy(), average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')

Using device: cpu
Epoch [1/500], Loss: 2.980665683746338, Train Acc: 0.0461,train F1-score:0.0721 Val Loss: 2.6787383556365967, Val Acc: 0.0569
Epoch [2/500], Loss: 2.7581398487091064, Train Acc: 0.1293,train F1-score:0.1937 Val Loss: 2.5343284606933594, Val Acc: 0.1278
Epoch [3/500], Loss: 2.587641954421997, Train Acc: 0.1816,train F1-score:0.2523 Val Loss: 2.414271354675293, Val Acc: 0.2356
Epoch [4/500], Loss: 2.4727580547332764, Train Acc: 0.2320,train F1-score:0.3046 Val Loss: 2.307441234588623, Val Acc: 0.4431
Epoch [5/500], Loss: 2.3718323707580566, Train Acc: 0.3377,train F1-score:0.3921 Val Loss: 2.208617925643921, Val Acc: 0.5716
Epoch [6/500], Loss: 2.29046368598938, Train Acc: 0.4080,train F1-score:0.4408 Val Loss: 2.1170504093170166, Val Acc: 0.6238
Epoch [7/500], Loss: 2.2236833572387695, Train Acc: 0.4305,train F1-score:0.4612 Val Loss: 2.026702880859375, Val Acc: 0.6212
Epoch [8/500], Loss: 2.1559863090515137, Train Acc: 0.4404,train F1-score:0.4614 Val Loss: 1.93777585

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the best GCN model
model.load_state_dict(torch.load('best_model.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train,edge_index_train).cpu().numpy()
    val_outputs = model(X_val,edge_index_val).cpu().numpy()
    test_outputs = model(X_test,edge_index_test).cpu().numpy()

# Train a decision tree classifier
decision_tree = DecisionTreeClassifier(max_depth=2)
decision_tree.fit(train_outputs, y_train.cpu().numpy())

# Predict labels using the decision tree
train_pred = decision_tree.predict(train_outputs)
val_pred = decision_tree.predict(val_outputs)
test_pred = decision_tree.predict(test_outputs)

# Evaluate decision tree performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (Decision Tree): {train_acc}')
print(f'Validation Accuracy (Decision Tree): {val_acc}')
print(f'Test Accuracy (Decision Tree): {test_acc}')

from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), test_pred, average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')


Training Accuracy (Decision Tree): 0.716714118238151
Validation Accuracy (Decision Tree): 0.7423025435073628
Test Accuracy (Decision Tree): 0.7091152815013405
Precision: 0.6278, Recall: 0.7091, F1-score: 0.6617


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Concatenate the GIN model output with the original feature matrices
X_train_combined = np.concatenate((X_train.cpu().numpy(), train_outputs), axis=1)
X_val_combined = np.concatenate((X_val.cpu().numpy(), val_outputs), axis=1)
X_test_combined = np.concatenate((X_test.cpu().numpy(), test_outputs), axis=1)

# Train a decision tree classifier
decision_tree = DecisionTreeClassifier(max_depth=9)
decision_tree.fit(X_train_combined, y_train.cpu().numpy())

# Predict labels using the decision tree
train_pred = decision_tree.predict(X_train_combined)
val_pred = decision_tree.predict(X_val_combined)
test_pred = decision_tree.predict(X_test_combined)

# Evaluate decision tree performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (Decision Tree): {train_acc}')
print(f'Validation Accuracy (Decision Tree): {val_acc}')
print(f'Test Accuracy (Decision Tree): {test_acc}')

from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), test_pred, average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')


Training Accuracy (Decision Tree): 0.8913079886116229
Validation Accuracy (Decision Tree): 0.8681392235609103
Test Accuracy (Decision Tree): 0.8659517426273459
Precision: 0.8549, Recall: 0.8660, F1-score: 0.8569


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.svm import SVC

In [ ]:
# Train an SVM classifier
svm_classifier = SVC(kernel='linear')  # You can specify different kernel functions (e.g., 'linear', 'poly', 'rbf', etc.)
svm_classifier.fit(train_outputs, y_train.cpu().numpy())

# Predict labels using the SVM classifier
train_pred = svm_classifier.predict(train_outputs)
val_pred = svm_classifier.predict(val_outputs)
test_pred = svm_classifier.predict(test_outputs)

# Evaluate SVM performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (SVM): {train_acc}')
print(f'Validation Accuracy (SVM): {val_acc}')
print(f'Test Accuracy (SVM): {test_acc}')
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), test_pred, average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')

Training Accuracy (SVM): 0.7816948584826662
Validation Accuracy (SVM): 0.8125836680053548
Test Accuracy (SVM): 0.7989276139410187
Precision: 0.7810, Recall: 0.7989, F1-score: 0.7775


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Train an SVM classifier
svm_classifier = SVC(kernel='linear')  # You can specify different kernel functions (e.g., 'linear', 'poly', 'rbf', etc.)
svm_classifier.fit(X_train_combined, y_train.cpu().numpy())

# Predict labels using the SVM classifier
train_pred = svm_classifier.predict(X_train_combined)
val_pred = svm_classifier.predict(X_val_combined)
test_pred = svm_classifier.predict(X_test_combined)

# Evaluate SVM performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (SVM): {train_acc}')
print(f'Validation Accuracy (SVM): {val_acc}')
print(f'Test Accuracy (SVM): {test_acc}')

from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), test_pred, average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')



Training Accuracy (SVM): 0.8542957628537934
Validation Accuracy (SVM): 0.8500669344042838
Test Accuracy (SVM): 0.8438337801608579
Precision: 0.8371, Recall: 0.8438, F1-score: 0.8338


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
